# ALERT SYSTEM

ADX: https://www.investopedia.com/articles/trading/07/adx-trend-indicator.asp

Cron Job: https://crontab.guru/
*/2 * * * *

In [2]:
import pandas as pd
import pandas_ta as ta
import ccxt, yfinance
import requests

In [3]:
exchange = ccxt.binance()

In [31]:
symbols = pd.read_csv("symbols.csv")
symbols = pd.DataFrame(symbols, columns = ["symbol"])
symbols = symbols["symbol"].values.tolist()
symbols

['ETH/USDT', 'BTC/USDT', 'SOL/USDT', 'AVAX/USDT']

In [35]:
for symbol in symbols:
    bars = exchange.fetch_ohlcv(symbol, timeframe = "5m", limit = 500)
    df = pd.DataFrame(bars, columns = ["time", "open", "high", "low", "close", "volume"])
    df['time'] = pd.to_datetime(df['time'], unit = 'ms')    
    
    # technical indicators
    adx = df.ta.adx()
    macd = df.ta.macd(fast = 14, slow = 28)
    rsi = df.ta.rsi()

    # create composite df
    df = pd.concat([df, adx, macd, rsi], axis = 1)

    # grab the last value
    last_row = df.iloc[-1]
    cap_time = last_row[0]

    # hook to discord channel
    WEBHOOK_URL = "https://discord.com/api/webhooks/982327105041875004/w_ume6BIxlHe3C9jgRUrUXWSFn3oiLcvpb2118vsNh8iZdRGHxD5LqYDer2cGwsjd_sL"

    # logic
    if last_row["ADX_14"] > 25:
        if last_row["DMP_14"] > last_row["DMN_14"]:
            message = f"{cap_time} STRONG UP TREND: {symbol}: The ADX is {last_row['ADX_14']: .2f} +DI {last_row['DMP_14']: .2f} -DI {last_row['DMN_14']: .2f}"
        if last_row["DMN_14"] > last_row["DMP_14"]:
            message = f"{cap_time} STRONG DOWN TREND:{symbol}: The ADX is {last_row['ADX_14']: .2f} +DI {last_row['DMP_14']: .2f} -DI {last_row['DMN_14']: .2f}"

        payload = {
            "username" : "alertbot",
            "content" : message
        }

        print(message)
        requests.post(WEBHOOK_URL, json = payload)

    if last_row["ADX_14"] < 25:
        message = f"{cap_time} no trend:{symbol}: The ADX is {last_row['ADX_14']: .2f} +DI {last_row['DMP_14']: .2f} -DI {last_row['DMN_14']: .2f}"

        payload = {
            "username" : "alertbot",
            "content" : message
        }
        
        print(message)
        requests.post(WEBHOOK_URL, json = payload)

2022-06-03 19:30:00 no trend:ETH/USDT: The ADX is  18.92 +DI  12.83 -DI  20.97
2022-06-03 19:30:00 no trend:BTC/USDT: The ADX is  6.77 +DI  20.07 -DI  19.50
2022-06-03 19:30:00 no trend:SOL/USDT: The ADX is  19.87 +DI  26.89 -DI  12.66
2022-06-03 19:30:00 no trend:AVAX/USDT: The ADX is  11.11 +DI  17.78 -DI  21.92
